Analysis of the booking table

In [4]:
from google.cloud import bigquery
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Now you can use the environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")


# Create a BigQuery client
client = bigquery.Client()

# Your BigQuery query
query = """
SELECT * FROM `big-mamma-project.base_analysis_tables.bookings_analysis`
"""

# Run the query and convert it to a pandas dataframe
bookings = client.query(query,location="EU").to_dataframe()

# Now df is a pandas DataFrame containing the results of the query
bookings

,id,order_id,date,site_id,status_simple,shift_category,venue_group_client_id,max_guests,duration,booked_by,first_name,last_name
0,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,None,2023-06-26,13,Complete,DAY,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,35,270,Staff,Privatisation,N
1,ahNzfnNldmVucm9vbXMtc2VjdXJlcigLEhtuaWdodGxvb3...,None,2023-01-12,12,Complete,DAY,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,1,15,Walk In,None,None
2,ahNzfnNldmVucm9vbXMtc2VjdXJlcigLEhtuaWdodGxvb3...,None,2023-09-30,83,Canceled,DINNER,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,1,15,Walk In,Lunch,None
3,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,None,2022-08-03,12,Complete,DAY,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,3,15,Booking Widget,Finn,L
4,ahNzfnNldmVucm9vbXMtc2VjdXJlcigLEhtuaWdodGxvb3...,None,2022-12-22,12,Canceled,DAY,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,1,15,Walk In,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1147033,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,A781468.49451,2023-08-21,81,Complete,DINNER,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,6,210,Booking Widget,Anshika,V
1147034,ahNzfnNldmVucm9vbXMtc2VjdXJlcigLEhtuaWdodGxvb3...,A700727.125703,2023-02-01,34,Complete,DINNER,ahNzfnNldmVucm9vbXMtc2VjdXJlcicLEhpuaWdodGxvb3...,2,210,Staff,Florence,E
1147035,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,"A781468.56373,A781468.56376",2023-09-20,81,Complete,LUNCH,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,9,210,Booking Widget,Andrew,W
1147036,ahNzfnNldmVucm9vbXMtc2VjdXJlciULEhtuaWdodGxvb3...,"A781468.62367,A781468.62370",2023-10-13,81,Complete,DINNER,ahNzfnNldmVucm9vbXMtc2VjdXJlciQLEhpuaWdodGxvb3...,9,210,Booking Widget,Hiral,T


## Bookings stats : origins, status, motivations..

Channels

In [8]:
#Booking channel per restaurant 
bookings_origin=bookings.groupby(['site_id','booked_by'],as_index=False)["id"].count()
bookings_origin=bookings_origin.rename(columns={'id':'count','site_id':'restaurant','booked_by':'booking_channel'})

#Creating a dict of total bookings types by restaurant
status ={}
for x in bookings_origin['restaurant'].unique():
    status[x]=bookings_origin.loc[bookings_origin['restaurant'] == x,'count'].sum()

#Function to calculate % of status in total participation for each restaurant 
x = 0
for y in bookings_origin['restaurant']:
        #using loc to append result to each different line of the df
        bookings_origin.loc[x, 'percent'] = round((bookings_origin.loc[x,"count"]/status[y])*100,2)
        x += 1       

bookings_origin.sort_values(by=['restaurant','count'],ascending=False)

,restaurant,booking_channel,count,percent
31,83,Booking Widget,27348,64.77
36,83,Walk In,5291,12.53
35,83,Sunday,4549,10.77
32,83,Google,1585,3.75
30,83,Bigmamma Website,1439,3.41
34,83,Staff,1408,3.33
33,83,Landing Page,600,1.42
24,81,Booking Widget,71839,69.53
29,81,Walk In,11029,10.67
28,81,Sunday,9991,9.67


Status per site

In [9]:
#Booking status per restaurant 
bookings_status=bookings.groupby(['site_id','status_simple'],as_index=False)["id"].count()
bookings_status=bookings_status.rename(columns={'id':'count','site_id':'restaurant','booked_by':'booking_channel'})

#Creating a dict of total bookings types by restaurant
status ={}
for x in bookings_status['restaurant'].unique():
    status[x]=bookings_status.loc[bookings_status['restaurant'] == x,'count'].sum()

#Function to calculate % of status in total participation for each restaurant 
x = 0
for y in bookings_status['restaurant']:
        #using loc to append result to each different line of the df
        bookings_status.loc[x, 'percent'] = round((bookings_status.loc[x,"count"]/status[y])*100,2)
        x += 1       

bookings_status.sort_values(by=['restaurant','count'],ascending=False)


,restaurant,status_simple,count,percent
25,83,Complete,28451,67.39
24,83,Canceled,8324,19.72
26,83,Incomplete,4651,11.02
27,83,No Show,794,1.88
21,81,Complete,68137,65.95
20,81,Canceled,23393,22.64
22,81,Incomplete,7684,7.44
23,81,No Show,4109,3.98
15,34,Complete,236030,67.20
14,34,Canceled,84237,23.98


In [10]:
bookings_origin.to_csv('bookings_origin.csv',index=False)
bookings_status.to_csv('bookings_status.csv',index=False )